# GJK

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import numpy as np

In [2]:
from pkg.tf_transform import *
from pkg.tf_robot import *
from pkg.constraint import *
from pkg.info import *
from pkg.tf_utils import *
from pkg.rotation_utils import *
from pkg.utils import *

In [3]:
import matplotlib.pyplot as plt
import time

## for test

In [4]:
def get_facepoints(verts, faces, N_fcs=20): # (fc, vtx, ax)
    global_save['verts'] = verts
    global_save['faces'] = faces
    return np.concatenate([np.array([verts[s1f-1] for s1f in faces]), np.zeros((N_fcs-len(faces),3,3))],axis=0)

def pad_vertex(verts, N_vtx):
    l_v = len(verts)
    assert l_v<N_vtx, "vertex more than N_vtx"
    return np.concatenate([verts, np.zeros((N_vtx-l_v,3))],axis=0).astype('float32')

In [5]:
gtimer = GlobalTimer()

In [6]:
N_vtx = 20

In [7]:
s1coords = np.loadtxt("s1.csv", delimiter=",")
s2coords = np.loadtxt("s2.csv", delimiter=",")

In [8]:
s1faces = np.loadtxt("s1face.csv", delimiter=",",dtype='int')
s2faces = np.loadtxt("s2face.csv", delimiter=",",dtype='int')

In [9]:
iterations = 6

In [10]:
s1verts = pad_vertex(s1coords.astype('float32'), N_vtx) # get_facepoints(s1coords, s1faces,N_fcs).astype('float32')
s2verts = pad_vertex(s2coords.astype('float32'), N_vtx) # get_facepoints(s2coords, s2faces,N_fcs).astype('float32')

In [11]:
S1Rot = np.array([
        [0.99847,0.016541,-0.052829,],
        [-0.014912,0.99941,0.031073,],
        [0.053311,-0.030238,0.99812,],
        ], dtype=np.float32)
S2Rot = np.array([
        [0.99752,0.065339,-0.026294,],
        [-0.063664,0.99616,0.060141,],
        [0.030123,-0.058318,0.99784,],
        ], dtype=np.float32)

In [12]:
s1points = tf.matmul(s1verts, np.transpose(S1Rot))+3
s2points = tf.matmul(s2verts, np.transpose(S2Rot))-3

In [13]:
FX1 = s1points
FX2 = s2points

## original_remove_face

In [92]:
global_save = {}
@tf.function
def getFarthestInDir(FX, v):
    dotted = K.sum(tf.multiply(FX,v),axis=-1)
    IdxSet = tf.expand_dims(tf.argmax(dotted, axis=-1),axis=-1)
    point=tf.gather(params = FX, indices=IdxSet)
    return point

@tf.function
def support(FX1, FX2, v):
    point1 = getFarthestInDir(FX1, v)
    point2 = getFarthestInDir(FX2, -v)
    return point1 - point2

@tf.function
def pickLineTF(v, FX1, FX2):
    b= support(FX2, FX1, v)
    a= support(FX2, FX1, -v)
    return a, b

@tf.function
def _loop_PickTriangle(a, b, c, flag, FX1, FX2):
    ab = b-a;
    ao = -a;
    ac = c-a;
    abc = tf.linalg.cross(ab,ac)
    abp = tf.linalg.cross(ab,abc)
    acp = tf.linalg.cross(abc,ac)
    abpo = K.sum(abp*ao) > 0
    acpo = K.sum(acp*ao) > 0
        
    a,b,c,flag = tf.case(
        [
            (abpo, lambda: (support(FX2, FX1,abp),a,b,False)),
            (acpo, lambda: (support(FX2, FX1,acp),a,c,False))
        ], default=lambda: (a,b,c,True))
    return a,b,c,flag, FX1, FX2

@tf.function
def _cond_PickTriangle(a, b, c, flag, FX1, FX2):
    return tf.logical_not(flag)
    

@tf.function
def PickTriangleTF(a, b, FX1, FX2, IterationAllowed=6):
    flag = False

    # First try:
    ab = b-a
    ao = -a
    v = tf.linalg.cross(tf.linalg.cross(ab,ao),ab) # v is perpendicular to ab pointing in the general direction of the origin
    c = b
    b = a
    a = support(FX2,FX1,v)
    a,b,c,flag, _, _ = tf.while_loop(
        _cond_PickTriangle, _loop_PickTriangle, (a,b,c,flag, FX1, FX2), parallel_iterations=10, maximum_iterations=IterationAllowed
    )

    return a, b, c, flag

@tf.function
def _loop_pickTetrahedron(a, b, c, d, flag, FX1, FX2):
    #Check the tetrahedron:
    ab = b-a
    ao = -a
    ac = c-a
    ad = d-a

    #We KNOW that the origin is not under the base of the tetrahedron based on
    #the way we picked a. So we need to check faces ABC, ABD, and ACD.

    #Normal to face of triangle
    abc = tf.linalg.cross(ab,ac)
    acd = tf.linalg.cross(ac,ad)
    adb = tf.linalg.cross(ad,ab)

    b,c,abc,flag = tf.cond(
        tf.greater(K.sum(abc*ao), 0), 
        lambda: (b,c,abc,flag), 
        lambda: tf.cond(
            tf.greater(K.sum(acd*ao), 0),
            lambda: (c,d,acd,flag),
            lambda: tf.cond(
                tf.greater(K.sum(adb*ao), 0),
                lambda: (d,b,adb,flag),
                lambda: (b,c,abc,True)
            )
        )
    )

    a,b,c,d = tf.cond(tf.greater(K.sum(abc*ao), 0), 
                        lambda: (support(FX2,FX1,abc),a,b,c), 
                        lambda: (support(FX2,FX1,-abc),a,c,b)
                       )
    return a, b, c, d, flag, FX1, FX2

@tf.function
def _cond_pickTetrahedron(a, b, c, d, flag, FX1, FX2):
    return tf.logical_not(flag)

@tf.function
def pickTetrahedronTF(a,b,c,FX1,FX2,IterationAllowed):
    flag = False
    ab = b-a
    ac = c-a

    # Normal to face of triangle
    abc = tf.linalg.cross(ab,ac)
    ao = -a

    a,b,c,d = tf.cond(tf.greater(K.sum(abc* ao), 0), 
                        lambda: (support(FX2,FX1,abc),a,b,c), 
                        lambda: (support(FX2,FX1,-abc),a,c,b)
                       )

    a, b, c, d, flag, _, _ = tf.while_loop(
        _cond_pickTetrahedron, _loop_pickTetrahedron, (a,b,c,d,flag, FX1, FX2), 
        parallel_iterations=10, maximum_iterations=IterationAllowed
    )   
    return a,b,c,d,flag

@tf.function
def test_collision(FX1, FX2, v, iterations):
    a, b = pickLineTF(v, FX2, FX1)
    a, b, c, flag = PickTriangleTF(a,b,FX2,FX1,iterations)
    a,b,c,d,flag = tf.cond(flag, # Only bother if we could find a viable triangle.
                           lambda: pickTetrahedronTF(a,b,c,FX2,FX1,iterations),
                           lambda: (a,b,c,c,flag))
    return flag

In [94]:
test_collision(FX1,FX2, v, iterations)

<tf.Tensor: shape=(), dtype=bool, numpy=False>

In [96]:
gtimer.reset()
v = np.array([0.8000, 0.5000, 1.0000], dtype=np.float32)
t1 = time.time()
for _ in range(1000):
    flag = test_collision(FX1, FX2, v, iterations)
    if flag:
        print("Collision")
t2 = time.time()
print("{} ms, while <1ms in matlab".format(t2-t1))
gtimer.print_time_log()

0.8917534351348877 ms, while <1ms in matlab


In [38]:
glog = {}
class CollisionLayer(layers.Layer):
    def __init__(self,*args, **kwargs):
        super(CollisionLayer, self).__init__(*args, **kwargs)
        
    # 변수를 만듭니다.
    def build(self, input_shape):
        pass

    # call 메서드가 그래프 모드에서 사용되면
    # training 변수는 텐서가 됩니다.
    @tf.function
    def call(self, inputs=None):
        v = tf.constant([0.8000, 0.5000, 1.0000])
        glog['FX1'] = inputs[0]
        glog['FX2'] = inputs[1]
        FX1, FX2 = inputs
        return test_collision(FX1, FX2, v)

In [39]:
glog

{}

In [40]:
# input = np.stack((FX1, FX2),axis=0)
input = (FX1, FX2)
CollisionLayer()(input)

TypeError: in user code:

    <ipython-input-38-58c41db4205d>:18 call  *
        return test_collision(FX1, FX2, v)
    <ipython-input-37-209f138bd3f2>:5 test_collision  *
        if flag == 1: # Only bother if we could find a viable triangle.
    /home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:1491 tensor_equals
        return gen_math_ops.equal(self, other, incompatible_shape_error=False)
    /home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/ops/gen_math_ops.py:3224 equal
        name=name)
    /home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:479 _apply_op_helper
        repr(values), type(values).__name__, err))

    TypeError: Expected bool passed to parameter 'y' of op 'Equal', got 1 of type 'int' instead. Error: Expected bool, got 1 of type 'int' instead.


In [122]:
tcl = layers.TimeDistributed(CollisionLayer())

In [122]:
tcl = layers.TimeDistributed(CollisionLayer())

In [122]:
tcl = layers.TimeDistributed(CollisionLayer())

In [124]:

inputs = np.stack([input]*5, axis=0)

In [105]:
tcl(inputs)

OperatorNotAllowedInGraphError: in user code:

    <ipython-input-95-f6ebf2b60800>:13 call  *
        FX1, FX2 = inputs
    /home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:561 __iter__
        self._disallow_iteration()
    /home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:554 _disallow_iteration
        self._disallow_when_autograph_enabled("iterating over `tf.Tensor`")
    /home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:532 _disallow_when_autograph_enabled
        " decorating it directly with @tf.function.".format(task))

    OperatorNotAllowedInGraphError: iterating over `tf.Tensor` is not allowed: AutoGraph did not convert this function. Try decorating it directly with @tf.function.


In [49]:
gtimer.print_time_log()

pickLine: 	299.0 ms/1000 = 0.299 ms
support: 	1520.0 ms/12000 = 0.127 ms
getFarthestInDir: 	1402.0 ms/24000 = 0.058 ms
getFarthestInDir0: 	513.0 ms/24000 = 0.021 ms
getFarthestInDir1: 	441.0 ms/24000 = 0.018 ms
getFarthestInDir2: 	372.0 ms/24000 = 0.016 ms
PickTriangle_init: 	158.0 ms/1000 = 0.158 ms
PickTriangle_loop: 	453.0 ms/2000 = 0.226 ms
PickTriangle_loop0: 	113.0 ms/3000 = 0.038 ms
pickTetrahedron_init: 	216.0 ms/1000 = 0.216 ms
pickTetrahedron_init0: 	7.0 ms/1000 = 0.007 ms
pickTetrahedron_init1: 	184.0 ms/1000 = 0.184 ms
pickTetrahedron_loop: 	2009.0 ms/6000 = 0.335 ms
pickTetrahedron_loop0: 	894.0 ms/6000 = 0.149 ms


## batch version original

In [54]:
@tf.function
def getFarthestInDir_batch(FX_batch, v_batch, batch_dims = 2):
    dotted = K.sum(FX_batch*v_batch,axis=-1)
    IdxSet = tf.expand_dims(tf.argmax(dotted, axis=-1),-1)
    point=tf.gather_nd(params = FX_batch, indices=IdxSet, batch_dims=batch_dims)
    return point

In [55]:
@tf.function
def getFarthestInDir(FX, v):
    dotted = K.sum(tf.multiply(FX,v),axis=-1)
    IdxSet = tf.expand_dims(tf.argmax(dotted, axis=-1),axis=-1)
    point=tf.gather(params = FX, indices=IdxSet)
    return point

## test batch

In [ ]:
N_sim = 50
N_col = 20

In [56]:
FX = FX1-FX2
v = np.array([[0.8000, 0.5000, 1.0000]],dtype=np.float32)

In [57]:
# FX = global_save['FX']
# v = global_save['v']
FX_batch = tf.stack([tf.stack([FX]*N_col, axis=0)]*N_sim,axis=0)
print(FX_batch.shape)
v_batch = tf.stack([tf.stack([v]*N_col, axis=0)]*N_sim,axis=0)
print(v_batch.shape)

(50, 20, 20, 3)
(50, 20, 1, 3)


In [58]:
gtimer.reset()
for _ in range(1000):
    gtimer.tic("FD_batch")
    point_batch = getFarthestInDir_batch(FX_batch, v_batch,1)
    gtimer.toc("FD_batch")
for _ in range(1000):
    gtimer.tic("FD_single")
    point = getFarthestInDir(FX, v)
    gtimer.toc("FD_single")
gtimer.print_time_log()

FD_batch: 	1764.0 ms/1000 = 1.764 ms
FD_single: 	60.0 ms/1000 = 0.06 ms


In [53]:
gtimer.reset()
for _ in range(1000):
    gtimer.tic("FD_batch")
    point_batch = getFarthestInDir_batch(FX_batch, v_batch,1)
    gtimer.toc("FD_batch")
for _ in range(1000):
    gtimer.tic("FD_single")
    point = getFarthestInDir(FX, v)
    gtimer.toc("FD_single")
gtimer.print_time_log()

FD_batch: 	599.0 ms/1000 = 0.599 ms
FD_single: 	212.0 ms/1000 = 0.212 ms


In [67]:
batch_dims = 2
dotted = K.sum(FX_batch*v_batch,axis=-1)
IdxSet = tf.expand_dims(tf.argmax(dotted, axis=-1),-1)
point=tf.gather_nd(params = FX_batch, indices=IdxSet, batch_dims=batch_dims)

In [68]:
point.shape

TensorShape([50, 20, 3])

In [61]:
point[0,0]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-2.2484524, -2.3406322, -3.0199368], dtype=float32)>

# Matthew Sheen version

In [43]:
import time
import collections
class GlobalTimer:
    def __init__(self, scale=1000):
        self.name_list = []
        self.ts_dict = {}
        self.time_dict = collections.defaultdict(lambda: 0)
        self.count_dict = collections.defaultdict(lambda: 0)
        self.scale = scale
        self.switch(True)
        
    def reset(self):
        self.name_list = []
        self.ts_dict = {}
        self.time_dict = collections.defaultdict(lambda: 0)
        self.count_dict = collections.defaultdict(lambda: 0)
        self.switch(True)
        
    def switch(self, onoff):
        self.__on = onoff
    
    def tic(self, name):
        if self.__on:
            if name not in self.name_list:
                self.name_list += [name]
            self.ts_dict[name] = time.time()
        
    def toc(self, name):
        if self.__on:
            self.time_dict[name] = self.time_dict[name]+(time.time() - self.ts_dict[name]) * self.scale
            self.count_dict[name] = self.count_dict[name] + 1
            
    def toctic(self, name_toc, name_tic):
        self.toc(name_toc)
        self.tic(name_tic)
        
    def print_time_log(self, names=None, timeunit="ms"):
        if names is None:
            names = self.name_list
        for name in names:
            print("{name}: \t{tot_T} {timeunit}/{tot_C} = {per_T} {timeunit}".format(
                name=name, tot_T=np.round(np.sum(self.time_dict[name])), tot_C=self.count_dict[name], 
                per_T= np.round(np.sum(self.time_dict[name])/self.count_dict[name], 3),
                timeunit=timeunit
            ))
        

In [44]:
gtimer = GlobalTimer()

In [84]:
global_save = {}
def getFarthestInDir(FX, v):
    global_save['FX'] = FX
    global_save['v'] = v
    gtimer.tic("getFarthestInDir")
    gtimer.tic("getFarthestInDir0")
    dotted = K.sum(tf.multiply(FX,v),axis=-1)
    gtimer.toctic("getFarthestInDir0", "getFarthestInDir1")
    rowIdxSet = tf.argmax(dotted, axis=-1)
    gtimer.toctic("getFarthestInDir1","getFarthestInDir2")
    maxInCol = K.max(dotted, axis=-1)
    gtimer.toctic("getFarthestInDir2", "getFarthestInDir3")
    colIdx = tf.expand_dims(tf.argmax(maxInCol, axis=-1),axis=-1)
    gtimer.toctic("getFarthestInDir3", "getFarthestInDir4")
    rowIdx=tf.gather(params = rowIdxSet, indices=colIdx)
    gtimer.toctic("getFarthestInDir4", "getFarthestInDir5")
    point=tf.gather_nd(params = FX, indices=tf.concat([colIdx,rowIdx],axis=-1))
    gtimer.toc("getFarthestInDir5")
    gtimer.toc("getFarthestInDir")
    return point

def get_facepoints(verts, faces): # (fc, vtx, ax)
    return np.array([verts[s1f-1] for s1f in faces])

def support(FX1, FX2, v):
    gtimer.tic("support")
    point1 = getFarthestInDir(FX1, v)
    point2 = getFarthestInDir(FX2, -v)
    gtimer.toc("support")
    return point1 - point2

def pickLine(v, FX1, FX2):
    gtimer.tic("pickLine")
    b= support(FX2, FX1, v)
    a= support(FX2, FX1, -v)
    gtimer.toc("pickLine")
    return a, b

def PickTriangle(a, b, FX1, FX2, IterationAllowed=6):
    flag = 0

    gtimer.tic("PickTriangle_init")
    # First try:
    ab = b-a
    ao = -a
    v = tf.linalg.cross(tf.linalg.cross(ab,ao),ab) # v is perpendicular to ab pointing in the general direction of the origin
    c = b
    b = a
    a = support(FX2,FX1,v)
    gtimer.toc("PickTriangle_init")
    for i in range(IterationAllowed):
        gtimer.tic("PickTriangle_loop")
        gtimer.tic("PickTriangle_loop0")
        ab = b-a;
        ao = -a;
        ac = c-a;
        abc = tf.linalg.cross(ab,ac)
        abp = tf.linalg.cross(ab,abc)
        acp = tf.linalg.cross(abc,ac)
        gtimer.toc("PickTriangle_loop0")
        if K.sum(abp*ao) > 0:
            c = b # Throw away the furthest point and grab a new one in the right direction
            b = a
            v = abp # cross(cross(ab,ao),ab);
        elif K.sum(acp*ao) > 0:
            b = a
            v = acp; # cross(cross(ac,ao),ac);

        else:
            flag = 1;
            break # We got a good one.
        a = support(FX2, FX1,v)
        gtimer.toc("PickTriangle_loop")

    return a, b, c, flag


def pickTetrahedron(a,b,c,FX1,FX2,IterationAllowed):
    gtimer.tic("pickTetrahedron_init")
    flag = 0
    ab = b-a
    ac = c-a

    # Normal to face of triangle
    gtimer.tic("pickTetrahedron_init0")
    abc = tf.linalg.cross(ab,ac)
    gtimer.toc("pickTetrahedron_init0")
    ao = -a

    gtimer.tic("pickTetrahedron_init1")
    if K.sum(abc* ao) > 0: # Above
        d = c
        c = b
        b = a

        v = abc
        a = support(FX2,FX1,v) # Tetrahedron new point
    else: # below
        d = b
        b = a
        v = -abc
        a = support(FX2,FX1,v) # Tetrahedron new point
    gtimer.toc("pickTetrahedron_init1")
    gtimer.toc("pickTetrahedron_init")

    for i in range(IterationAllowed): #Allowing 10 tries to make a good tetrahedron.
        gtimer.tic("pickTetrahedron_loop")
        gtimer.tic("pickTetrahedron_loop0")
        #Check the tetrahedron:
        ab = b-a
        ao = -a
        ac = c-a
        ad = d-a

        #We KNOW that the origin is not under the base of the tetrahedron based on
        #the way we picked a. So we need to check faces ABC, ABD, and ACD.

        #Normal to face of triangle
        abc = tf.linalg.cross(ab,ac)

        if K.sum(abc*ao) > 0: #Above triangle ABC
            pass
            # No need to change anything, we'll just iterate again with this face as
            # default.
        else:
            acd = tf.linalg.cross(ac,ad) # Normal to face of triangle

            if K.sum(acd*ao) > 0 : # Above triangle ACD
                # Make this the new base triangle.
                b = c
                c = d
                ab = ac
                ac = ad            
                abc = acd
            elif K.sum(acd*ao) < 0:
                adb = tf.linalg.cross(ad,ab) #Normal to face of triangle

                if K.sum(adb*ao) > 0: #Above triangle ADB
                    # Make this the new base triangle.
                    c = b
                    b = d
                    ac = ab
                    ab = ad
                    abc = adb
                else:
                    flag = 1
                    break # It's inside the tetrahedron.
        gtimer.toc("pickTetrahedron_loop0")

        #try again:
        if K.sum(abc*ao) > 0: #Above
            d = c
            c = b
            b = a    
            v = abc
            a = support(FX2,FX1,v) #Tetrahedron new point
        else: #below
            d = b;
            b = a;
            v = -abc;
            a = support(FX2,FX1,v) #Tetrahedron new point
        gtimer.toc("pickTetrahedron_loop")
    return a,b,c,d,flag

# Test

In [85]:
gtimer = GlobalTimer()

In [86]:
s1coords = np.loadtxt("s1.csv", delimiter=",")
s2coords = np.loadtxt("s2.csv", delimiter=",")

In [87]:
s1faces = np.loadtxt("s1face.csv", delimiter=",",dtype='int')
s2faces = np.loadtxt("s2face.csv", delimiter=",",dtype='int')

In [88]:
iterations = 6

In [89]:
s1verts = get_facepoints(s1coords, s1faces)
s2verts = get_facepoints(s2coords, s2faces)

In [90]:
S1Rot = [
        [0.99847,0.016541,-0.052829,],
        [-0.014912,0.99941,0.031073,],
        [0.053311,-0.030238,0.99812,],
        ]
S2Rot = [
        [0.99752,0.065339,-0.026294,],
        [-0.063664,0.99616,0.060141,],
        [0.030123,-0.058318,0.99784,],
        ]

In [142]:
s1points = tf.matmul(s1verts[:,:,:], np.transpose(S1Rot))+3
s2points = tf.matmul(s2verts[:,:,:], np.transpose(S2Rot))-3

In [143]:
FX1 = s1points
FX2 = s2points

In [93]:
gtimer.reset()
v = np.array([0.8000, 0.5000, 1.0000])
t1 = time.time()
for _ in range(1000):
    a, b = pickLine(v, FX2, FX1)
    a, b, c, flag = PickTriangle(a,b,FX2,FX1,iterations)
    if flag == 1: # Only bother if we could find a viable triangle.
        a,b,c,d,flag = pickTetrahedron(a,b,c,FX2,FX1,iterations)
t2 = time.time()
print("{} ms, while <1ms in matlab".format(t2-t1))

4.702171325683594 ms, while <1ms in matlab


In [147]:
v = np.array([0.8000, 0.5000, 1.0000])
a,b = pickLine(v, FX2, FX1)
a,b

(<tf.Tensor: shape=(3,), dtype=float64, numpy=array([4.38314668, 5.93255553, 3.86193882])>,
 <tf.Tensor: shape=(3,), dtype=float64, numpy=array([7.61685332, 6.06744447, 8.13806118])>)

In [148]:
a, b, c, flag = PickTriangle(a,b,FX2,FX1,iterations)
a, b, c

(<tf.Tensor: shape=(3,), dtype=float64, numpy=array([3.4942057 , 5.39867512, 6.97876186])>,
 <tf.Tensor: shape=(3,), dtype=float64, numpy=array([3.60530508, 5.3333286 , 4.8797155 ])>,
 <tf.Tensor: shape=(3,), dtype=float64, numpy=array([4.17041999, 3.65601581, 6.01532408])>)

In [149]:
pickTetrahedron(a,b,c,FX2,FX1,iterations)

(<tf.Tensor: shape=(3,), dtype=float64, numpy=array([4.17041999, 3.65601581, 6.01532408])>,
 <tf.Tensor: shape=(3,), dtype=float64, numpy=array([4.17041999, 3.65601581, 6.01532408])>,
 <tf.Tensor: shape=(3,), dtype=float64, numpy=array([3.4942057 , 5.39867512, 6.97876186])>,
 <tf.Tensor: shape=(3,), dtype=float64, numpy=array([3.60530508, 5.3333286 , 4.8797155 ])>,
 0)

# Matthew Sheen numpy version

In [841]:
def getFarthestInDir(FX, v):
    dotted = np.sum(FX*v,axis=-1)
    rowIdxSet = np.argmax(dotted, axis=-1)
    maxInCol = np.max(dotted, axis=-1)
    colIdx = np.argmax(maxInCol, axis=-1)
    rowIdx = rowIdxSet[colIdx]
    point = FX[colIdx,rowIdx]
    return point


def get_facepoints(verts, faces): # (fc, vtx, ax)
    return np.array([verts[s1f-1] for s1f in faces])

def support(FX1, FX2, v):
    point1 = getFarthestInDir(FX1, v)
    point2 = getFarthestInDir(FX2, -v)
    return point1 - point2

def pickLine(v, FX1, FX2):
    b= support(FX2, FX1, v)
    a= support(FX2, FX1, -v)
    return a, b

def PickTriangle(a, b, FX1, FX2, IterationAllowed=6):
    flag = 0

    # First try:
    ab = b-a
    ao = -a
    v = np.cross(np.cross(ab,ao),ab) # v is perpendicular to ab pointing in the general direction of the origin
    c = b
    b = a
    a = support(FX2,FX1,v)
    for i in range(IterationAllowed):
        ab = b-a;
        ao = -a;
        ac = c-a;
        abc = np.cross(ab,ac)
        abp = np.cross(ab,abc)
        acp = np.cross(abc,ac)
        if np.sum(abp*ao) > 0:
            c = b # Throw away the furthest point and grab a new one in the right direction
            b = a
            v = abp # cross(cross(ab,ao),ab);
        elif np.sum(acp*ao) > 0:
            b = a
            v = acp; # cross(cross(ac,ao),ac);

        else:
            flag = 1;
            break # We got a good one.
        a = support(FX2, FX1,v)

    return a, b, c, flag


def pickTetrahedron(a,b,c,FX1,FX2,IterationAllowed):
    flag = 0
    ab = b-a
    ac = c-a

    # Normal to face of triangle
    abc = np.cross(ab,ac)
    ao = -a

    if np.sum(abc* ao) > 0: # Above
        d = c
        c = b
        b = a

        v = abc
        a = support(FX2,FX1,v) # Tetrahedron new point
    else: # below
        d = b
        b = a
        v = -abc
        a = support(FX2,FX1,v) # Tetrahedron new point

    for i in range(IterationAllowed): #Allowing 10 tries to make a good tetrahedron.
        #Check the tetrahedron:
        ab = b-a
        ao = -a
        ac = c-a
        ad = d-a

        #We KNOW that the origin is not under the base of the tetrahedron based on
        #the way we picked a. So we need to check faces ABC, ABD, and ACD.

        #Normal to face of triangle
        abc = np.cross(ab,ac)

        if np.sum(abc*ao) > 0: #Above triangle ABC
            pass
            # No need to change anything, we'll just iterate again with this face as
            # default.
        else:
            acd = np.cross(ac,ad) # Normal to face of triangle

            if np.sum(acd*ao) > 0 : # Above triangle ACD
                # Make this the new base triangle.
                b = c
                c = d
                ab = ac
                ac = ad            
                abc = acd
            elif np.sum(acd*ao) < 0:
                adb = np.cross(ad,ab) #Normal to face of triangle

                if np.sum(adb*ao) > 0: #Above triangle ADB
                    # Make this the new base triangle.
                    c = b
                    b = d
                    ac = ab
                    ab = ad
                    abc = adb
                else:
                    flag = 1
                    break # It's inside the tetrahedron.

        #try again:
        if np.sum(abc*ao) > 0: #Above
            d = c
            c = b
            b = a    
            v = abc
            a = support(FX2,FX1,v) #Tetrahedron new point
        else: #below
            d = b;
            b = a;
            v = -abc;
            a = support(FX2,FX1,v) #Tetrahedron new point
    return a,b,c,d,flag

# Test

In [842]:
s1coords = np.loadtxt("s1.csv", delimiter=",")
s2coords = np.loadtxt("s2.csv", delimiter=",")

In [843]:
s1faces = np.loadtxt("s1face.csv", delimiter=",",dtype='int')
s2faces = np.loadtxt("s2face.csv", delimiter=",",dtype='int')

In [844]:
iterations = 6

In [845]:
s1verts = get_facepoints(s1coords, s1faces)
s2verts = get_facepoints(s2coords, s2faces)

In [846]:
S1Rot = [
        [0.99847,0.016541,-0.052829,],
        [-0.014912,0.99941,0.031073,],
        [0.053311,-0.030238,0.99812,],
        ]
S2Rot = [
        [0.99752,0.065339,-0.026294,],
        [-0.063664,0.99616,0.060141,],
        [0.030123,-0.058318,0.99784,],
        ]

In [847]:
s1points = tf.matmul(s1verts[:,:,:], np.transpose(S1Rot))+3
s2points = tf.matmul(s2verts[:,:,:], np.transpose(S2Rot))-3

In [848]:
FX1 = s1points
FX2 = s2points

In [851]:
t1 = time.time()
for _ in range(1000):
    v = np.array([0.8000, 0.5000, 1.0000])
    a, b = pickLine(v, FX2, FX1)
    a, b, c, flag = PickTriangle(a,b,FX2,FX1,iterations)
    if flag == 1: # Only bother if we could find a viable triangle.
        a,b,c,d,flag = pickTetrahedron(a,b,c,FX2,FX1,iterations)
t2 = time.time()
print("{} ms, while <1ms in matlab".format(t2-t1))

7.197966814041138 ms, while <1ms in matlab
